In [1]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime as dt
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 22
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
def get_len(worksheet, df):
    lengths = []
    for idx, col in enumerate(df):
        series = df[col]
        max_len = max((series.astype(str).map(len).max(),
                       len(str(series.name)))) + 2
        lengths.append(max_len)
    return lengths

In [3]:
def create_report(path):
    xlsx_list = sorted(glob.glob(f'{path}*.xlsx'))
    output = pd.DataFrame()
    for name in xlsx_list:
        df = pd.read_excel(name, sheet_name = 'сводная таблица',
                           index_col = [0, 1],
                           usecols = [0, 1, 2, 3, 4])
        df.drop(index='Все', inplace=True)
        sums = df.groupby(level = 0).sum().drop(columns='Процент неликвидов')
        df.drop(columns = ['Всего новинок', 'Всего неликвидов'], inplace=True)
        df['Процент неликвидов'] /= 100
        sums['ИТОГ % НЕЛИКВИДОВ'] = sums['Всего неликвидов'] / sums['Всего новинок']
        sums.rename(columns={'Всего новинок':'ИТОГ ШТ. НОВИНОК',
                             'Всего неликвидов':'ИТОГ ШТ. НЕЛИКВИДОВ'}, inplace=True)
        sums['ИТОГ ШТ. НОВИНОК'] = sums['ИТОГ ШТ. НОВИНОК'].astype(str) + ' шт.'
        sums['ИТОГ ШТ. НЕЛИКВИДОВ'] = sums['ИТОГ ШТ. НЕЛИКВИДОВ'].astype(str) + ' шт.'
        sums = sums.stack().to_frame(name='Процент неликвидов')
        sums['dummy'] = 1
        period = pd.concat([df, sums])
        period.rename(columns={'Процент неликвидов':name[-15:-8]}, inplace=True)
        output = pd.concat([output, period], axis=1)
    output['sorter'] = output['dummy'].sum(axis=1)
    output = output.sort_values(by=['Редакция', 'sorter']).drop(columns=['dummy', 'sorter'])
    return output.reset_index()

In [4]:
# Write pivot table to file
output = create_report('input/АСТ/')
output
writer = pd.ExcelWriter(f'omut.xlsx', engine='xlsxwriter') 
output.to_excel(writer, index=False, sheet_name='сводная таблица', na_rep='')

,Редакция,Подразделение,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
0,507 Департамент Планета Детства,Аванта,0.22963,0.139535,0.110236,0.132812,0.145161,0.115702,0.119658,0.098214,0.117117,0.17757,0.237705,0.216783
1,507 Департамент Планета Детства,Астрель СПб. Малыш (Обучающая и развивающая ли...,0.268293,0.243243,0.297297,0.4,0.55,0.375,0.232558,0.307692,0.351351,0.384615,0.393939,0.317073
2,507 Департамент Планета Детства,Астрель СПб. Малыш (Художественная литература),0.122807,0.127273,0.172414,0.166667,0.138462,0.186441,0.12069,0.137931,0.142857,0.232143,0.290909,0.287879
3,507 Департамент Планета Детства,Вилли-Винки,0.113208,0.092784,0.07,0.132653,0.13,0.19,0.075472,0.12381,0.212121,0.330097,0.4,0.408163
4,507 Департамент Планета Детства,Интеджер. Аванта,0.068182,0.051282,0.055556,0.078947,0.081081,0.02381,0.028571,0.078947,0.05,0.06383,0.104167,0.085106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,511 Департамент Прикладная литература и межизд...,Кладезь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201149
61,511 Департамент Прикладная литература и межизд...,ОГИЗ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356164
62,511 Департамент Прикладная литература и межизд...,ИТОГ ШТ. НОВИНОК,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1287 шт.
63,511 Департамент Прикладная литература и межизд...,ИТОГ ШТ. НЕЛИКВИДОВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400 шт.


In [41]:
# Init objects
workbook = writer.book
max_row, max_col = output.shape
# editorials = ['100 - Редакция "Комильфо"',
#               '101 - Редакция № 1',
#               '102 - Редакция № 2',
#               '120 - Редакция № 5']
editorials = ['507 Департамент Планета Детства',
              '509 Департамент Художественной литературы',
              '510 Департамент Межиздат',
              '511 Департамент Прикладная литература']

In [42]:
# Main chart 
# Get chart data
chart1_data = output.loc[lambda df: df['Подразделение'].isin(['ИТОГ % НЕЛИКВИДОВ']), :]
chart1_data = chart1_data.loc[lambda df: df['Редакция'].isin(editorials), :]

chart1 = workbook.add_chart({'type': 'line'})
for i in chart1_data.index:
    chart1.add_series({
        'name': ['сводная таблица', i+1, 0, i+1, 0],
        'values': ['сводная таблица', i+1, 2, i+1, max_col-1],
        'marker': {'type': 'diamond', 'size': 5, 'fill': {'color': 'black'}},
        'categories' : ['сводная таблица', 0, 2, 0, max_col-1]})
chart1.set_legend({'position':'bottom',
                   'font': {'size': 9}})
chart1.set_x_axis({'minor_gridlines': {'visible': True}})
chart1.set_y_axis({'minor_gridlines': {'visible': True}})
worksheet = workbook.add_chartsheet('Редакции')
worksheet.set_chart(chart1)

In [43]:
# Specific charts
# Get chart data
for i, edtr in enumerate(editorials):
    if i == 0:
        continue
    elif i == 1:
        part1 = output.loc[lambda df: df['Редакция'].isin([editorials[1]]), :]
        part1.drop(part1.tail(3).index, inplace=True)
        part2 = output.loc[lambda df: df['Редакция'].isin([editorials[0]]), :]
        part2.drop(part2.tail(3).index, inplace=True)
        spec_chart_data = pd.concat([part1, part2])
    else:
        spec_chart_data = output.loc[lambda df: df['Редакция'].isin([edtr]), :]
        spec_chart_data.drop(spec_chart_data.tail(3).index, inplace=True)
    spec_chart = workbook.add_chart({'type': 'line'})
    for i in spec_chart_data.index:
        spec_chart.add_series({
            'name': ['сводная таблица', i+1, 1, i+1, 1],
            'values': ['сводная таблица', i+1, 2, i+1, max_col-1],
            'marker': {'type': 'diamond', 'size': 5, 'fill': {'color': 'black'}},
            'categories' : ['сводная таблица', 0, 2, 0, max_col-1]})
    spec_chart.set_legend({'position':'bottom',
                           'font': {'size': 8}})
    spec_chart.set_x_axis({'minor_gridlines': {'visible': True}})
    spec_chart.set_y_axis({'minor_gridlines': {'visible': True}})
    worksheet = workbook.add_chartsheet(str(edtr)[:3])
    worksheet.set_chart(spec_chart)

In [44]:
# Init formats
bld_brdr = workbook.add_format({'bold': True,
                                'border': 1})
bld_brdr_clr = workbook.add_format({'bold': True,
                                     'border': 1,
                                     'bg_color': '#FABF8F',
                                     'num_format': '0.0%'})
bld_brdr_clr_l = workbook.add_format({'bold': True,
                                     'border': 1,
                                     'bg_color': '#FDE9D9',
                                     'num_format': '0.0%'})
perc = workbook.add_format({'num_format': '0.0%'})

In [45]:
# Cell formating
worksheet = writer.sheets['сводная таблица']
lens = get_len(worksheet, output)
itog_idx = output.loc[output['Подразделение'] == 'ИТОГ ШТ. НЕЛИКВИДОВ'].index
main_itog_idx = chart1_data.index
worksheet.set_column(2, max_col, 8, perc)
worksheet.set_column('A:A', lens[0], bld_brdr)
worksheet.set_column('B:B', lens[1], bld_brdr)
for idx in itog_idx:
    if idx+1 in main_itog_idx:
        clr = bld_brdr_clr
    else:
        clr = bld_brdr_clr_l
    worksheet.set_row(idx, None, clr)
    worksheet.set_row(idx+1, None, clr)
    worksheet.set_row(idx+2, None, clr)

0

0

0

0

0

...

0

0

0

0

0

In [5]:
writer.close()